<a href="https://colab.research.google.com/github/apoorva-aa/IR_Project_2023/blob/main/Lexical_text_simplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers
!pip install sentence-transformers
!pip install wordfreq
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 931.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=25d0bba204cdce5c0c7acaa4d927af8e3177af416555abcde6720a6d2528ae4c
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Looking in indexes: https://pypi.

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import torch
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForMaskedLM
from wordfreq import zipf_frequency
from sklearn.metrics.pairwise import cosine_similarity
import textstat
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
bert_similarity_model = SentenceTransformer('bert-base-nli-mean-tokens')

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [50]:
import string
import itertools
def sentence_complexity(sentence):
  wpt = WordPunctTokenizer()
  tokens = wpt.tokenize(sentence)
  complexity_sum = 0
  num_tokens = len(tokens)
  for i in range(num_tokens):
    complexity_sum+= zipf_frequency(tokens[i], 'en')
  return complexity_sum
def user_interface_function(text):
  def get_top_k_predictions(input_string, k,tokenizer,model) -> str:
    tokenized_inputs = tokenizer(input_string, return_tensors="tf")
    outputs = model(tokenized_inputs["input_ids"])
    top_k_indices = tf.math.top_k(outputs.logits, 7).indices[0].numpy()
    decoded_output = tokenizer.batch_decode(top_k_indices)
    mask_token = tokenizer.encode(tokenizer.mask_token)[1:-1]
    mask_index = np.where(tokenized_inputs['input_ids'].numpy()[0]==mask_token)[0][0]

    decoded_output_words = decoded_output[mask_index]
    return decoded_output_words    
  text = text.lower()
  wpt = WordPunctTokenizer()
  tokens = wpt.tokenize(text)
  complex_words = []
  index_of_complex_word = []
  for i in range(len(tokens)):
    if(textstat.difficult_words(tokens[i]) == 1):
      complex_words.append(tokens[i])
      index_of_complex_word.append(i)
  sentence1_transformed = bert_similarity_model.encode(text)
  best_2_word_list = []
  if(len(tokens) < 6):
    pass
    #to do
  else:
    for j in range(len(complex_words)):
      text_with_mask = text.replace(complex_words[j], '[MASK]')
      predictedWords = get_top_k_predictions(text_with_mask,10,tokenizer,model)
      predictedWords = predictedWords.split()
      word = ''
      remove_words_list = []
      for i in range(len(predictedWords)):
        if(zipf_frequency(predictedWords[i], 'en') < zipf_frequency(complex_words[j], 'en')):
          remove_words_list.append(i)
        else:
          for letter in predictedWords[i]:
            if(letter in string.punctuation):
              remove_words_list.append(i)
              break
      remove_words_list.sort(reverse=True)
      for i in range(len(remove_words_list)):
        predictedWords.pop(remove_words_list[i])
      similarity = []
      print(predictedWords)
      for i in range(len(predictedWords)):
        if(complex_words[j] == predictedWords[i]):
          similarity.append(0)
        simpler_text = text.replace(complex_words[j], predictedWords[i])
        sentence2_transformed = bert_similarity_model.encode(simpler_text)
        similarity_val = cosine_similarity([sentence1_transformed],[sentence2_transformed])[0][0]
        similarity.append(similarity_val)
      if(len(similarity) == 0):
        best_2_word_list.append([complex_words[j]])
      else:
        best_word = predictedWords[similarity.index(max(similarity))]
        predictedWords.pop(similarity.index(max(similarity)))
        similarity.pop(similarity.index(max(similarity)))
        if(len(predictedWords) != 0):
          best_word_2 = predictedWords[similarity.index(max(similarity))]
          best_2_word_list.append([best_word,best_word_2])
        else:
          best_2_word_list.append([complex_words[j]])
    similarity_arr = []
    possible_combinations = list(itertools.product(*best_2_word_list))
    print(best_2_word_list)
    possible_combinations_complexity = []
    simpler_text_arr = []
    for i in range(len(possible_combinations)):
      simpler_text = text
      words_complexity = 0
      for j in range(len(complex_words)):
        simpler_text = simpler_text.replace(complex_words[j], possible_combinations[i][j])
        words_complexity += zipf_frequency(tokens[i], 'en')
      simpler_text_arr.append(simpler_text)
      possible_combinations_complexity.append(words_complexity)
      sentence2_transformed = bert_similarity_model.encode(simpler_text)
      similarity_val = cosine_similarity([sentence1_transformed],[sentence2_transformed])[0][0]
      similarity_arr.append(similarity_val)
    indexed_similarity_arr = list(enumerate(similarity_arr))
    sorted_sim_arr = sorted(indexed_similarity_arr, key=lambda x: x[1],reverse = True)
    indexed_possible_combinations_complexity = list(enumerate(possible_combinations_complexity))
    sorted_complex_arr = sorted(indexed_possible_combinations_complexity, key=lambda x: x[1],reverse = True)
    # print(sorted_sim_arr)
    # print(sorted_complex_arr)
    mydict_sim = {}
    mydict_complex = {}
    for i in range(len(sorted_sim_arr)):
      mydict_sim[sorted_sim_arr[i][0]] = i + 1
      mydict_complex[sorted_complex_arr[i][0]] = i + 1
    # print(mydict_sim)
    # print(mydict_complex)
    min_rank = 100000000000000000
    required_index = 0
    for i in range(len(sorted_sim_arr)):
      rank_val = mydict_sim[i] * mydict_complex[i]
      # print(rank_val)
      if(min_rank > rank_val):
        min_rank = rank_val
        required_index = i
    # print(required_index)
    new_sentence = text
    for j in range(len(complex_words)):
        new_sentence = new_sentence.replace(complex_words[j], possible_combinations[required_index][j])
    return new_sentence

In [ ]:
input_text = ""
simplified_text = user_interface_function(input_text)

In [49]:
simplified_text

'the door to his former apartment was locked but the police knocked down the door with full force and arrested him'

In [52]:
print(sentence_complexity(input_text),sentence_complexity(simplified_text))

123.61 126.33
